# Assignment 10
Bill Henderson

CSCI-E-116

Spring 2025

# Imports

In [40]:
import pandas as pd
from fredapi import Fred
from yaml import safe_load
from arch.unitroot import DFGLS
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import *
import random
import warnings

random.seed(42)
warnings.filterwarnings('ignore')

# Data Ingress


In [22]:
fred_key = safe_load(open('/Users/hendwi1/env.yaml'))['fred_key']

In [23]:
fred = Fred(api_key=fred_key)
mortgage = fred.get_series("MORTGAGE30US")
treasury = fred.get_series("DGS10")

In [24]:
mortgage

1971-04-02    7.33
1971-04-09    7.31
1971-04-16    7.31
1971-04-23    7.31
1971-04-30    7.29
              ... 
2025-03-27    6.65
2025-04-03    6.64
2025-04-10    6.62
2025-04-17    6.83
2025-04-24    6.81
Length: 2822, dtype: float64

In [25]:
treasury

1962-01-02    4.06
1962-01-03    4.03
1962-01-04    3.99
1962-01-05    4.02
1962-01-08    4.03
              ... 
2025-04-21    4.42
2025-04-22    4.41
2025-04-23    4.40
2025-04-24    4.32
2025-04-25    4.29
Length: 16519, dtype: float64

## Convert them to the monthly frequency and starts in April 1971

In [26]:
# chose to get the mean of montly values since it seemed to make the most sense. min, max, first, and last could be considered also.
mort_monthly = mortgage.resample('M').mean().to_period('M')
mort_monthly = mort_monthly[mort_monthly.index >= '1971-04']
mort_monthly

1971-04    7.3100
1971-05    7.4250
1971-06    7.5300
1971-07    7.6040
1971-08    7.6975
            ...  
2024-12    6.7150
2025-01    6.9580
2025-02    6.8425
2025-03    6.6500
2025-04    6.7250
Freq: M, Length: 649, dtype: float64

In [27]:
treas_monthly = treasury.resample('M').mean().to_period('M')
treas_monthly = treas_monthly[treas_monthly.index >= '1971-04']
treas_monthly

1971-04    5.833810
1971-05    6.392500
1971-06    6.522273
1971-07    6.729524
1971-08    6.582727
             ...   
2024-12    4.391429
2025-01    4.629048
2025-02    4.451053
2025-03    4.280476
2025-04    4.292778
Freq: M, Length: 649, dtype: float64

# Calculate the error correction (ec = m30 – t10)

In [28]:
ec = mort_monthly - treas_monthly
ec.describe()

count    649.000000
mean       1.752871
std        0.518425
min        0.533864
25%        1.430238
50%        1.657619
75%        1.975714
max        4.851667
dtype: float64

# Using the ADF test and DF-GLS test to determine that m30, t10, and ec are stationary or not.

In [34]:
def adf_test(timeseries, varname):
    print(f"{varname.capitalize()} Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)
    print('\n\n')

In [36]:

data = {
    'mortgage': mort_monthly,
    'treasury': treas_monthly,
    'ec': ec
}

_ = [adf_test(v,k) for k,v in data.items()]


Mortgage Results of Dickey-Fuller Test:
Test Statistic                  -1.338153
p-value                          0.611541
#Lags Used                       3.000000
Number of Observations Used    645.000000
Critical Value (1%)             -3.440529
Critical Value (5%)             -2.866031
Critical Value (10%)            -2.569162
dtype: float64



Treasury Results of Dickey-Fuller Test:
Test Statistic                  -1.450028
p-value                          0.558071
#Lags Used                      12.000000
Number of Observations Used    636.000000
Critical Value (1%)             -3.440674
Critical Value (5%)             -2.866095
Critical Value (10%)            -2.569196
dtype: float64



Ec Results of Dickey-Fuller Test:
Test Statistic                  -3.386703
p-value                          0.011421
#Lags Used                      20.000000
Number of Observations Used    628.000000
Critical Value (1%)             -3.440806
Critical Value (5%)             -2.866153
Critical V

In [38]:
df_gls = {k: DFGLS(v).pvalue for k,v in data.items()}
df_gls

{'mortgage': np.float64(0.17264041330860252),
 'treasury': np.float64(0.14002867079723463),
 'ec': np.float64(0.010691011197406071)}

Appears both the ADF and DF-GLS both say that there's sufficient evidence that the mortgage and treasury data is not stationary, but the EC calculation is stationary. This is due to high p values on both tests for mortgage and treasury, but a very low p value for both tests with the EC calculated data.

# Vector Error Correction Model (VECM)

In [57]:
df = pd.concat([mort_monthly, treas_monthly], axis=1)
df.columns = ['mortgage', 'treasury']
df

,mortgage,treasury
1971-04,7.3100,5.833810
1971-05,7.4250,6.392500
1971-06,7.5300,6.522273
1971-07,7.6040,6.729524
1971-08,7.6975,6.582727
...,...,...
2024-12,6.7150,4.391429
2025-01,6.9580,4.629048
2025-02,6.8425,4.451053
2025-03,6.6500,4.280476


In [60]:
lag_order = select_order(df, maxlags = 20)
print(lag_order)

<statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 7, BIC -> 2, FPE -> 7, HQIC ->  2>


In [64]:
rank_test = select_coint_rank(df, 0, 3, method="trace",
                              signif=0.05)
rank_test.rank

1

In [65]:
mod = VECM(
    df,
    k_ar_diff=lag_order.aic,
    coint_rank=rank_test.rank
)
res = mod.fit()
res.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
L1.mortgage,0.0333,0.052,0.644,0.520,-0.068,0.135
L1.treasury,0.5751,0.041,13.882,0.000,0.494,0.656
L2.mortgage,-0.2415,0.053,-4.547,0.000,-0.346,-0.137
L2.treasury,0.0769,0.047,1.633,0.103,-0.015,0.169
L3.mortgage,-0.0353,0.054,-0.649,0.516,-0.142,0.071
L3.treasury,0.1431,0.047,3.018,0.003,0.050,0.236
L4.mortgage,-0.0003,0.054,-0.006,0.995,-0.106,0.106
L4.treasury,-0.0091,0.047,-0.193,0.847,-0.101,0.083
L5.mortgage,-0.0616,0.054,-1.138,0.255,-0.168,0.044
L5.treasury,0.1070,0.047,2.260,0.024,0.014,0.200


From the above summary of the VECM, both ec values for mortgage and treasury have p values that are too high to assume they are statistically significant.